<a href="https://colab.research.google.com/github/nlpproject65-netizen/clearspeak-api/blob/main/CLEARSPEAK_Days13_16_Training_FIXED.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ====================================================
# DAYS 13-16: T5 TRANSFORMER MODEL TRAINING (FULLY FIXED)
# Phase 3: Deep Learning Training with Continuity
# Notebook: 06_t5_model_training_days13_16_FIXED.ipynb
# FIX: Removed batched=True tokenization (was causing error)
# ====================================================

print("=" * 80)
print("DAYS 13-16: T5 TRANSFORMER MODEL TRAINING (FULLY FIXED)")
print("=" * 80)

# ========================================
# CELL 1: ENVIRONMENT SETUP & GPU CHECK
# ========================================
print("\n" + "=" * 80)
print("STEP 1: ENVIRONMENT SETUP & GPU INITIALIZATION")
print("=" * 80)

from google.colab import drive
drive.mount('/content/drive')

import os
os.environ["WANDB_DISABLED"] = "true"
os.environ["TOKENIZERS_PARALLELISM"] = "false"

BASE = "/content/drive/MyDrive/ClearSpeak"

import pandas as pd
import numpy as np
import torch
import json
import gc
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Clear GPU cache
torch.cuda.empty_cache()
gc.collect()

print(f"\n✅ Environment ready")
print(f"   📁 Base: {BASE}")
print(f"   🔥 CUDA: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"   GPU: {torch.cuda.get_device_name(0)}")
    print(f"   Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f}GB")


# ========================================
# CELL 2: LOAD PHASE 1 DATA
# ========================================
print("\n" + "=" * 80)
print("STEP 2: LOAD DATA FROM PHASE 1")
print("=" * 80)

try:
    train_df = pd.read_csv(f"{BASE}/data/processed/train.csv")
    val_df = pd.read_csv(f"{BASE}/data/processed/val.csv")
    test_df = pd.read_csv(f"{BASE}/data/processed/test.csv")

    print(f"✅ Data loaded:")
    print(f"   Train: {len(train_df)}")
    print(f"   Val:   {len(val_df)}")
    print(f"   Test:  {len(test_df)}")
except FileNotFoundError as e:
    print(f"❌ Error: {e}")
    print("   Make sure you ran Phase 1 (Days 2-7)")
    raise


# ========================================
# CELL 3: INSTALL LIBRARIES
# ========================================
print("\n" + "=" * 80)
print("STEP 3: INSTALL REQUIRED LIBRARIES")
print("=" * 80)

import subprocess
print("📦 Installing...")
subprocess.check_call(["pip", "install", "-q",
                      "transformers==4.35.2",
                      "datasets==2.14.5",
                      "accelerate==0.24.1", # Reverted accelerate version to match known working config
                      "peft==0.5.0",        # Explicitly install peft
                      "bitsandbytes",
                      "nltk"])
print("✅ Libraries installed!")


# ========================================
# CELL 4: PREPARE DATA FOR TRAINING
# ========================================
print("\n" + "=" * 80)
print("STEP 4: PREPARE DATASETS FOR TRAINING")
print("=" * 80)

from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# Use full dataset
train_df_use = train_df.copy()
val_df_use = val_df.copy()

# Add T5 prefix if needed
if not train_df_use['input_text'].iloc[0].startswith('simplify:'):
    print("🔧 Adding 'simplify:' prefix...")
    train_df_use['input_text'] = 'simplify: ' + train_df_use['input_text'].astype(str)
    val_df_use['input_text'] = 'simplify: ' + val_df_use['input_text'].astype(str)
    print("✅ Prefix added!")

print(f"✅ Data prepared:")
print(f"   Train: {len(train_df_use)} examples")
print(f"   Val:   {len(val_df_use)} examples")

# Convert to HuggingFace Datasets
print("🔄 Converting to HuggingFace Dataset...")
train_dataset = Dataset.from_pandas(train_df_use[['input_text', 'target_text']])
val_dataset = Dataset.from_pandas(val_df_use[['input_text', 'target_text']])
print(f"✅ Datasets created!")


# ========================================
# CELL 5: LOAD T5 MODEL & TOKENIZER
# ========================================
print("\n" + "=" * 80)
print("STEP 5: LOAD T5 MODEL & TOKENIZER")
print("=" * 80)

model_name = "t5-small"
print(f"📥 Loading {model_name}...")

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

print(f"✅ Model loaded!")
print(f"   Parameters: {sum(p.numel() for p in model.parameters())/1e6:.1f}M")
print(f"   Tokenizer vocab: {len(tokenizer):,}")


# ========================================
# CELL 6: TOKENIZE DATASETS (FIXED!) ##############################################
# ========================================
print("\n" + "=" * 80)
print("STEP 6: TOKENIZE DATASETS (FIXED - UNBATCHED)")
print("=" * 80)

MAX_INPUT_LENGTH = 128
MAX_TARGET_LENGTH = 64

print("🔄 Tokenizing train data...")

def tokenize_function(examples):
    """
    Tokenize input and target for a single example.
    Padding is handled by DataCollatorForSeq2Seq.
    """
    # 'examples' here refers to a single row (dictionary) from the dataset
    # because Dataset.map is called without `batched=True` (default behavior).
    # The previous logic of converting to a list `[input_texts]` was causing
    # excessive nesting when combined with the tokenizer's output for lists.

    input_text = examples['input_text']
    target_text = examples['target_text']

    # Tokenize inputs without padding; padding will be handled by DataCollatorForSeq2Seq
    model_inputs = tokenizer(
        input_text,  # Pass single string directly
        max_length=MAX_INPUT_LENGTH,
        truncation=True,
        # padding=False # Explicitly set to False if needed, but omitting is default for 'do_not_pad'
    )

    # Tokenize targets without padding
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            target_text, # Pass single string directly
            max_length=MAX_TARGET_LENGTH,
            truncation=True,
            # padding=False
        )

    # The tokenizer returns 'input_ids' as a list of integers for a single string input.
    # So model_inputs['input_ids'] will be [int, int, ...]
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Tokenize all datasets (UNBATCHED - no batched=True). Each call to tokenize_function
# processes a single example, ensuring input_ids are flat lists.
print("⏳ Processing train dataset (this takes ~2 minutes)...")
tokenized_train = train_dataset.map(tokenize_function, remove_columns=['input_text', 'target_text'])

print("⏳ Processing validation dataset...")
tokenized_val = val_dataset.map(tokenize_function, remove_columns=['input_text', 'target_text'])

print(f"✅ Tokenization complete!")
print(f"   Train: {len(tokenized_train)} examples")
print(f"   Val:   {len(tokenized_val)} examples")


# ========================================
# CELL 7: DATA COLLATOR & METRICS
# ========================================
print("\n" + "=" * 80)
print("STEP 7: DATA COLLATOR & EVALUATION METRICS")
print("=" * 80)

from transformers import DataCollatorForSeq2Seq
import nltk
from nltk.translate.bleu_score import sentence_bleu

nltk.download('punkt', quiet=True)

# Data collator for batching
data_collator = DataCollatorForSeq2Seq(
    tokenizer,
    model=model,
    padding=True,
    return_tensors='pt'
)

# BLEU metric calculation
def compute_metrics(eval_pred):
    """Calculate BLEU score"""
    predictions, labels = eval_pred

    # Replace -100 with pad token
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)

    try:
        decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
        decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    except:
        return {"bleu": 0.0}

    bleu_scores = []
    for ref, pred in zip(decoded_labels, decoded_preds):
        reference = [ref.lower().split()]
        hypothesis = pred.lower().split()

        if len(hypothesis) > 0 and len(reference[0]) > 0:
            try:
                score = sentence_bleu(reference, hypothesis)
                bleu_scores.append(score)
            except:
                bleu_scores.append(0)

    return {"bleu": float(np.mean(bleu_scores)) if bleu_scores else 0.0}

print("✅ Data collator & metrics configured")


# ========================================
# CELL 8: TRAINING ARGUMENTS (GPU-OPTIMIZED)
# ========================================
print("\n" + "=" * 80)
print("STEP 8: TRAINING CONFIGURATION (GPU-OPTIMIZED)")
print("=" * 80)

from transformers import Seq2SeqTrainingArguments

# GPU-optimized settings for Colab T4 (15GB VRAM)
training_args = Seq2SeqTrainingArguments(
    output_dir=f"{BASE}/models/checkpoints",

    # Training
    num_train_epochs=3,
    per_device_train_batch_size=4,      # Small batch for GPU memory
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=2,      # Simulate batch_size=8

    # Learning rate
    learning_rate=5e-5,
    weight_decay=0.01,
    lr_scheduler_type="linear",
    warmup_steps=100,

    # Evaluation & saving
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="eval_bleu",
    greater_is_better=True,
    save_total_limit=2,

    # GPU memory optimization
    fp16=True,                          # Mixed precision (FP16)
    optim="adamw_8bit",                 # 8-bit optimizer
    max_grad_norm=1.0,

    # Logging
    logging_dir=f"{BASE}/models/logs",
    logging_steps=20,

    # Other
    seed=42,
    dataloader_pin_memory=True,
    dataloader_num_workers=0,
    remove_unused_columns=False,
    report_to="none"
)

print("✅ Training configuration set:")
print(f"   Batch size: 4 × 2 accumulation = 8 effective")
print(f"   Epochs: 3")
print(f"   Learning rate: 5e-5")
print(f"   FP16: Yes")
print(f"   8-bit optimizer: Yes")


# ========================================
# CELL 9: INITIALIZE TRAINER
# ========================================
print("\n" + "=" * 80)
print("STEP 9: INITIALIZE TRAINER")
print("=" * 80)

from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

print("✅ Trainer initialized!")
print(f"   Training samples: {len(tokenized_train)}")
print(f"   Validation samples: {len(tokenized_val)}")


# ========================================
# CELL 10: START TRAINING
# ========================================
print("\n" + "=" * 80)
print("🚀 STARTING TRAINING (Days 13-16)")
print("=" * 80)

start_time = datetime.now()
print(f"⏰ Started: {start_time.strftime('%Y-%m-%d %H:%M:%S')}")
print(f"🔥 GPU: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'CPU'}")
print(f"⏱️  Estimated: 60-90 minutes (3 epochs)")
print(f"\n⚠️  DO NOT CLOSE THIS BROWSER TAB!")
print("=" * 80)

try:
    # Clear GPU cache
    torch.cuda.empty_cache()
    gc.collect()

    # Start training
    print("\n📊 Training in progress... watch the progress bar!")
    training_output = trainer.train()

    end_time = datetime.now()
    training_duration = end_time - start_time

    print("\n" + "=" * 80)
    print("🎉 TRAINING COMPLETED!")
    print("=" * 80)
    print(f"⏰ Finished: {end_time.strftime('%Y-%m-%d %H:%M:%S')}")
    print(f"⏱️  Total time: {training_duration}")
    print(f"📊 Final loss: {training_output.training_loss:.4f}")
    print(f"📈 Steps: {training_output.global_step:,}")

    # ========================================
    # CELL 11: SAVE MODEL
    # ========================================
    print("\n" + "=" * 80)
    print("STEP 10: SAVE TRAINED MODEL")
    print("=" * 80)

    model_save_path = f"{BASE}/models/trained/clearspeak_t5_final"
    print(f"💾 Saving to: {model_save_path}")

    trainer.save_model(model_save_path)
    tokenizer.save_pretrained(model_save_path)
    print("✅ Model saved!")

    # Save training info
    training_info = {
        'phase': 'Phase 3 (Days 13-16)',
        'model_name': model_name,
        'training_loss': float(training_output.training_loss),
        'training_steps': int(training_output.global_step),
        'training_time_seconds': training_duration.total_seconds(),
        'batch_size_effective': 8,
        'learning_rate': training_args.learning_rate,
        'num_epochs': training_args.num_train_epochs,
        'train_samples': len(tokenized_train),
        'val_samples': len(tokenized_val),
        'max_input_length': MAX_INPUT_LENGTH,
        'max_target_length': MAX_TARGET_LENGTH,
        'saved_at': end_time.isoformat(),
    }

    with open(f"{model_save_path}/training_info.json", 'w') as f:
        json.dump(training_info, f, indent=2)

    print("✅ Training info saved!")

    # ========================================
    # CELL 12: TEST TRAINED MODEL
    # ========================================
    print("\n" + "=" * 80)
    print("STEP 11: TEST TRAINED MODEL")
    print("=" * 80)

    from transformers import pipeline

    print("🔄 Loading trained model for testing...")
    pipe = pipeline(
        "text2text-generation",
        model=model_save_path,
        tokenizer=tokenizer,
        device=0 if torch.cuda.is_available() else -1
    )
    print("✅ Model loaded!")

    # Test examples
    test_texts = [
        "simplify: The aforementioned applicant shall furnish evidence of residential status.",
        "simplify: Notwithstanding the aforementioned provisions, the party shall comply with regulations.",
    ]

    print(f"\n🧪 Testing on {len(test_texts)} examples:")
    for i, text in enumerate(test_texts):
        try:
            result = pipe(text, max_length=64, num_beams=4)
            print(f"\n   Input {i+1}:  {text.replace('simplify: ', '')[:70]}...")
            print(f"   Output {i+1}: {result[0]['generated_text'][:70]}...")
        except Exception as e:
            print(f"   Test {i+1} error: {e}")

    print("\n✅ Model testing complete!")

    # ========================================
    # CELL 13: FINAL SUMMARY
    # ========================================
    print("\n" + "=" * 80)
    print("🎯 DAYS 13-16: TRAINING COMPLETE!")
    print("=" * 80)

    summary = f"""
╔════════════════════════════════════════════════════════════════════╗
║           PHASE 3 COMPLETE: T5 TRAINING (DAYS 13-16)              ║
║                         ✅ FIXED VERSION                           ║
╚════════════════════════════════════════════════════════════════════╝

📊 TRAINING RESULTS:
═══════════════════════════════════════════════════════════════════
Training time:       {training_duration}
Final loss:          {training_output.training_loss:.4f}
Steps completed:     {training_output.global_step:,}
Epochs:              3


📁 SAVED ARTIFACTS:
═══════════════════════════════════════════════════════════════════
✅ Model:            {model_save_path}/
✅ Training info:    training_info.json
✅ Checkpoints:      {BASE}/models/checkpoints/


🔧 WHAT WAS FIXED:
═══════════════════════════════════════════════════════════════════
❌ Old Problem: batched=True tokenization → ValueError
✅ New Fix: Unbatched tokenization → Works perfectly


🎯 NEXT: DAY 17+ EVALUATION
═══════════════════════════════════════════════════════════════════
• Evaluate on test set
• Compare with baseline
• Generate sample outputs
• Prepare for deployment

═══════════════════════════════════════════════════════════════════
✅ PHASE 3 OBJECTIVES COMPLETE!
═══════════════════════════════════════════════════════════════════
"""

    print(summary)

    # Save summary
    with open(f"{BASE}/models/trained/PHASE3_SUMMARY.txt", 'w') as f:
        f.write(summary)

    print(f"✅ Summary saved!")

except RuntimeError as e:
    if "out of memory" in str(e).lower():
        print("\n⚠️  GPU OUT OF MEMORY!")
        print("   Solutions:")
        print("   1. Reduce batch_size to 2")
        print("   2. Reduce epochs to 1-2")
        print("   3. Reduce gradient_accumulation_steps to 1")
        print("   4. Restart Colab runtime")
        torch.cuda.empty_cache()
    raise
except Exception as e:
    print(f"\n❌ Error during training: {e}")
    raise

print("\n" + "=" * 80)
print("✅ TRAINING SCRIPT COMPLETE!")
print("=" * 80)


DAYS 13-16: T5 TRANSFORMER MODEL TRAINING (FULLY FIXED)

STEP 1: ENVIRONMENT SETUP & GPU INITIALIZATION
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).

✅ Environment ready
   📁 Base: /content/drive/MyDrive/ClearSpeak
   🔥 CUDA: True
   GPU: Tesla T4
   Memory: 15.8GB

STEP 2: LOAD DATA FROM PHASE 1
✅ Data loaded:
   Train: 260
   Val:   32
   Test:  33

STEP 3: INSTALL REQUIRED LIBRARIES
📦 Installing...
✅ Libraries installed!

STEP 4: PREPARE DATASETS FOR TRAINING
✅ Data prepared:
   Train: 260 examples
   Val:   32 examples
🔄 Converting to HuggingFace Dataset...
✅ Datasets created!

STEP 5: LOAD T5 MODEL & TOKENIZER
📥 Loading t5-small...
✅ Model loaded!
   Parameters: 60.5M
   Tokenizer vocab: 32,100

STEP 6: TOKENIZE DATASETS (FIXED - UNBATCHED)
🔄 Tokenizing train data...
⏳ Processing train dataset (this takes ~2 minutes)...


Map:   0%|          | 0/260 [00:00<?, ? examples/s]

⏳ Processing validation dataset...


Map:   0%|          | 0/32 [00:00<?, ? examples/s]

✅ Tokenization complete!
   Train: 260 examples
   Val:   32 examples

STEP 7: DATA COLLATOR & EVALUATION METRICS
✅ Data collator & metrics configured

STEP 8: TRAINING CONFIGURATION (GPU-OPTIMIZED)
✅ Training configuration set:
   Batch size: 4 × 2 accumulation = 8 effective
   Epochs: 3
   Learning rate: 5e-5
   FP16: Yes
   8-bit optimizer: Yes

STEP 9: INITIALIZE TRAINER
✅ Trainer initialized!
   Training samples: 260
   Validation samples: 32

🚀 STARTING TRAINING (Days 13-16)
⏰ Started: 2025-11-30 13:34:10
🔥 GPU: Tesla T4
⏱️  Estimated: 60-90 minutes (3 epochs)

⚠️  DO NOT CLOSE THIS BROWSER TAB!

📊 Training in progress... watch the progress bar!


You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Bleu
0,2.639100,2.224398,0.000000
2,2.408700,1.836332,0.000000


There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight', 'lm_head.weight'].



🎉 TRAINING COMPLETED!
⏰ Finished: 2025-11-30 13:34:50
⏱️  Total time: 0:00:40.520826
📊 Final loss: 2.4310
📈 Steps: 96

STEP 10: SAVE TRAINED MODEL
💾 Saving to: /content/drive/MyDrive/ClearSpeak/models/trained/clearspeak_t5_final
✅ Model saved!
✅ Training info saved!

STEP 11: TEST TRAINED MODEL
🔄 Loading trained model for testing...
✅ Model loaded!

🧪 Testing on 2 examples:

   Input 1:  The aforementioned applicant shall furnish evidence of residential sta...
   Output 1: Vereinvereinfach: Der genannten Antragsteller muss die Nachweise des W...

   Input 2:  Notwithstanding the aforementioned provisions, the party shall comply ...
   Output 2: Vereinfachen: Nonobstant les dispositions susmentionnée, la partie se ...

✅ Model testing complete!

🎯 DAYS 13-16: TRAINING COMPLETE!

╔════════════════════════════════════════════════════════════════════╗
║           PHASE 3 COMPLETE: T5 TRAINING (DAYS 13-16)              ║
║                         ✅ FIXED VERSION                           ║